In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from statistic import Statistic
pd.set_option('display.max_colwidth', -1)

In [2]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [3]:
# ky9
path_data = '/mnt/files/workdata/work/merchants/merchant_32_ky9_2020-05-12_white_visa/'
df = pd.read_csv(path_data + '02_pure_data/ky9_for_resolve.csv', dtype=str)

In [4]:
df.head()

,order_id,type,status,date,processed,amount,currency,card_holder,email,phone,...,ip,project_id,reported,status_chargeback,date_cb_only,card_masked,card_expire,amount_in_currency,date_only,id
0,41799680781449564,charge,false,2018-12-19 22:33:23,2018-12-19 22:33:27,52.95,USD,Jennifer Myers,jenn-mye@hotmail.com,+6024818120,...,195.39.171.227,5a2df766c7321c3b48791d979bdad3ce5ce2f8b9,NaN,charged,2018-12-19,447494******6650,2019-03,52.95,2018-12-19,447494******6650_2019-03
1,41799692345652096,charge,false,2018-12-19 22:34:08,2018-12-19 22:34:12,21.18,USD,Brady Marske,marske-brady7053@yahoo.com,+3235272757,...,99.101.8.179,5a2df766c7321c3b48791d979bdad3ce5ce2f8b9,NaN,charged,2018-12-19,434256******3600,2022-11,21.18,2018-12-19,434256******3600_2022-11
2,41799707414784518,charge,false,2018-12-19 22:35:07,2018-12-19 22:35:10,26.48,USD,Christopher Scala,chsca8515@me.com,+2674232337,...,100.34.37.252,5a2df766c7321c3b48791d979bdad3ce5ce2f8b9,NaN,charged,2018-12-19,464733******3963,2020-04,26.48,2018-12-19,464733******3963_2020-04
3,41799716768741310,charge,false,2018-12-19 22:35:43,2018-12-19 22:35:47,26.48,USD,Blake Mitrovik,mitrovikblake@msn.com,+7152525225,...,174.198.11.206,5a2df766c7321c3b48791d979bdad3ce5ce2f8b9,NaN,charged,2018-12-19,437303******7009,2022-09,26.48,2018-12-19,437303******7009_2022-09
4,41799744497264522,charge,false,2018-12-19 22:37:31,2018-12-19 22:37:36,22.24,USD,Curry Paraspolo,paraspolocur@outlook.com,+3863141489,...,172.58.43.103,5a2df766c7321c3b48791d979bdad3ce5ce2f8b9,NaN,charged,2018-12-19,423223******9977,2022-11,22.24,2018-12-19,423223******9977_2022-11


In [5]:
df.dtypes

order_id              object
type                  object
status                object
date                  object
processed             object
amount                object
currency              object
card_holder           object
email                 object
phone                 object
address               object
city_from_order       object
country               object
zip                   object
ip                    object
project_id            object
reported              object
status_chargeback     object
date_cb_only          object
card_masked           object
card_expire           object
amount_in_currency    object
date_only             object
id                    object
dtype: object

In [6]:
df.shape

(4312751, 24)

In [7]:
min(df.date_only)

'2018-12-19'

In [8]:
max(df.date_only)

'2020-05-12'

In [9]:
df.date_only = pd.to_datetime(df.date_only)

In [10]:
df.date_cb_only = pd.to_datetime(df.date_cb_only)

In [11]:
start_date = min(df.date_only)
start_date

Timestamp('2018-12-19 00:00:00')

In [12]:
end_date = max(df.date_only)
end_date

Timestamp('2020-05-12 00:00:00')

In [13]:
diff = end_date - start_date
diff_days = diff.days
diff_days

510

In [14]:
days_in_test = int(diff_days * 0.2)
start_test = end_date - timedelta(days=days_in_test)
start_test

Timestamp('2020-01-31 00:00:00')

In [15]:
retreat_days_from_end = int(diff_days * 0.025)
end_test = end_date - timedelta(days=retreat_days_from_end)
end_test

Timestamp('2020-04-30 00:00:00')

In [16]:
retreat_days_from_start = 20
start_teach = start_date + timedelta(days=retreat_days_from_start)
start_teach

Timestamp('2019-01-08 00:00:00')

In [17]:
retreat_days_from_test_start = int(diff_days * 0.025)
end_teach = start_test - timedelta(days=retreat_days_from_test_start)
end_teach

Timestamp('2020-01-19 00:00:00')

In [18]:
start_teach

Timestamp('2019-01-08 00:00:00')

In [19]:
end_teach

Timestamp('2020-01-19 00:00:00')

In [20]:
start_test

Timestamp('2020-01-31 00:00:00')

In [22]:
end_test

Timestamp('2020-04-30 00:00:00')

In [31]:
# если не устраивают даты start_teach, end_teach, start_test, end_test их можно перезадать явно( пример внизу)
# start_teach = datetime.strptime("2020-01-01", '%Y-%m-%d')

In [23]:
good_statuses = Statistic.get_dt_good(df).status.unique()
good_status = good_statuses[0] 
good_status

'false'

In [24]:
mask_teach = (df.date_only < start_test) & ( (df.status.isin(Statistic.BAD_STATUSES)) | (df.date_only < end_teach ))
teach = df[mask_teach].copy()

In [25]:
Statistic.get_table(teach, 'status')

Counter({'false': 3258906, 'true': 48953})

In [26]:
teach.status = np.where( (teach.status.isin(Statistic.BAD_STATUSES)) & (teach.date_cb_only > start_test), good_status, teach.status)

In [27]:
Statistic.get_table(teach, 'status')

Counter({'false': 3263023, 'true': 44836})

In [28]:
mask_test = (df.date_only > start_test) &  ((df.status.isin(Statistic.BAD_STATUSES)) | (df.date_only < end_test))
test = df[mask_test]

In [29]:
Statistic.get_table(test, 'status')

Counter({'false': 810544, 'true': 7480})

In [30]:
Statistic.get_cb_rate(teach)

1.36

In [31]:
Statistic.get_cb_rate(test)

0.91

In [32]:
Statistic.get_cb_rate_amount(teach)

1.87

In [35]:
Statistic.get_cb_rate_amount(test)

1.5

In [44]:
teach_name = 'teach_' + start_teach.strftime('%Y-%m-%d') + '_' + end_teach.strftime('%Y-%m-%d') + '.csv'
test_name = 'test_' + start_test.strftime('%Y-%m-%d') + '_' + end_test.strftime('%Y-%m-%d') + '.csv'
teach_name

'teach_2019-01-08_2020-01-19.csv'

In [45]:
test_name

'test_2020-01-31_2020-04-30.csv'

In [49]:
path_exp = path_data + '04_experiments/ex_01_description/'
path_teach = path_exp + teach_name
path_test = path_exp + test_name

In [50]:
drop_columns = ['type', 'amount_in_currency']

In [53]:
teach = teach.drop(columns=drop_columns)
test = test.drop(columns=drop_columns)

In [54]:
list(teach)

['order_id',
 'status',
 'date',
 'processed',
 'amount',
 'currency',
 'card_holder',
 'email',
 'phone',
 'address',
 'city_from_order',
 'country',
 'zip',
 'ip',
 'project_id',
 'reported',
 'status_chargeback',
 'date_cb_only',
 'card_masked',
 'card_expire',
 'date_only',
 'id']

In [ ]:
#teach.to_csv(path_teach, index=False, quoting=csv.QUOTE_ALL)
#test.to_csv(path_teach, index=False, quoting=csv.QUOTE_ALL)